[https://www.kaggle.com/code/rakibulhasanshaon69/yoga-pose-detection/edit](http://)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageFile

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten,
    GlobalAveragePooling2D, Input, Multiply, Reshape
)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.applications import ResNet101, EfficientNetB4
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import tensorflow_hub as hub
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
def clean_invalid_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except (IOError, SyntaxError):
                print(f"Removing corrupted file: {file_path}")
                os.remove(file_path)


data= "/kaggle/input/yoga-pose-image-classification-dataset/dataset"
classes= sorted([folder for folder in os.listdir(data) if os.path.isdir(os.path.join(data, folder))])
clean_invalid_images(data)

In [ ]:
def visualize_images(data, classes, num_images=5):
    plt.figure(figsize=(15, len(classes) * 3)) 
    
    for idx, cls in enumerate(classes):
        class_path = os.path.join(data, cls)
        images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

        for i, img_path in enumerate(images[:num_images]):
            plt.subplot(len(classes), num_images, idx * num_images + i + 1)
            img = Image.open(img_path)
            plt.imshow(img)
            plt.axis('off')
            if i == 0:
                plt.title(cls, fontsize=12)

    plt.tight_layout()
    plt.show()
visualize_images(data, classes, num_images=5)

In [ ]:
for class_name in os.listdir(data):
    class_path = os.path.join(data, class_name)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            with Image.open(image_path) as img:
                print(f"Image: {image_name}, Class: {class_name}, Shape: {img.size}")
            break  

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data,
    image_size=(224, 224), 
    batch_size=32,  
    label_mode='int'  #  'categorical' or 'binary' 
)
#'categorical' ব্যবহার করবেন যদি one-hot encoding চাচ্ছেন (বিশেষত যখন আপনার মডেলটি categorical crossentropy loss ফাংশন ব্যবহার করবে)।
#'binary' ব্যবহার করবেন যদি বাইনারি ক্লাসিফিকেশন করছেন (যেমন, 2টি ক্লাসের মধ্যে নির্বাচন)।
#'int' ব্যবহার করবেন যদি মাল্টিক্লাস ক্লাসিফিকেশন করতে চান এবং লেবেলগুলি একটি সংখ্যা হিসেবে চান (যেমন, 0 থেকে 106)।

dataset = dataset.map(lambda x, y: (x / 255.0, y))

iterator = iter(dataset)
images, labels = next(iterator)
print(images.shape, labels.shape)

# spliting data

In [ ]:
train_size = int(len(dataset) * 0.75)
val_size = int(len(dataset) * 0.15)
test_size = int(len(dataset) * 0.10)
val_size

In [ ]:
train = dataset.take(train_size) 
val = dataset.skip(train_size).take(val_size) 
test = dataset.skip(train_size + val_size).take(test_size) 

# EfficientNet

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest'
)

input_tensor = Input(shape=(224, 224, 3))
base_model = EfficientNetB4(weights="imagenet", include_top=False, input_tensor=input_tensor)
base_model.trainable = True  

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation="relu", kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)  
x = Dense(107, activation="softmax")(x) 

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer=SGD(learning_rate=1e-3, momentum=0.9, nesterov=True), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
logdir = 'logs/efficientnet_b4'
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)

hist = model.fit(
    train,
    epochs=20,
    validation_data=val,
    callbacks=[tensorboard_callback]
)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

# ResNet101

In [ ]:
input_tensor = Input(shape=(224, 224, 3))
base_model = ResNet101(weights="imagenet", include_top=False, input_tensor=input_tensor)
base_model.trainable = True

gap = GlobalAveragePooling2D()(base_model.output)
se = Dense(256, activation="relu")(gap)
se = Dense(base_model.output_shape[-1], activation="sigmoid")(se)
se = Reshape((1, 1, base_model.output_shape[-1]))(se)
se = Multiply()([base_model.output, se])
x = GlobalAveragePooling2D()(se)
x = Dense(1024, activation="relu", kernel_regularizer=l2(1e-4))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(107, activation="softmax", kernel_regularizer=l2(1e-4))(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=1e-4), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
logdir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()


# Vision Transformar(base)

In [ ]:
# vit_model = hub.KerasLayer("https://tfhub.dev/sayakpaul/vit_b16_fe/1", trainable=True)
# input_tensor = Input(shape=(224, 224, 3))

# x = vit_model(input_tensor)
# x = Dense(1024, activation="relu")(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)  # Regularization
# x = Dense(107, activation="softmax")(x)

# model = Model(inputs=input_tensor, outputs=x)

# model.compile(optimizer=Adam(learning_rate=3e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
# model.summary()

# Vision Transformar(advance)

In [ ]:
# import tensorflow_hub as hub
# import tensorflow as tf
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Multiply, Reshape
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l2

# vit_model = hub.KerasLayer("https://tfhub.dev/google/vit_base_patch16_224/1", trainable=True)

# input_tensor = Input(shape=(224, 224, 3))
# x = vit_model(input_tensor)

# se = Dense(512, activation="relu")(x)
# se = Dense(x.shape[-1], activation="sigmoid")(se)
# se = Multiply()([x, se])

# x = Dense(1024, activation="relu", kernel_regularizer=l2(1e-4))(se)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu", kernel_regularizer=l2(1e-4))(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)
# x = Dense(107, activation="softmax", kernel_regularizer=l2(1e-4))(x)

# model = Model(inputs=input_tensor, outputs=x)
# model.compile(optimizer=Adam(learning_rate=3e-5), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
# model.summary()